In [1]:
import requests
import json
import pandas as pd
import time
#import convex as cx
import tmqa2 as tmqa

Autosaving every 180 seconds





Using TensorFlow backend.


Loading the params file
Input encoding {'o': 2, '{': 3, '.': 4, 'J': 5, '0': 6, '1': 7, '<': 8, 'B': 9, 'd': 10, '£': 11, 'e': 12, '6': 13, '!': 14, 'O': 15, 'M': 16, 'X': 17, 'f': 18, 't': 19, 'C': 20, 'V': 21, 'z': 22, 'K': 23, '\\': 24, '9': 25, 'P': 26, 'S': 27, '/': 28, '₹': 29, 'F': 30, 'G': 31, '=': 32, '8': 33, ')': 34, '+': 35, ']': 36, 'U': 37, "'": 38, '"': 39, 'g': 40, 'N': 41, 'r': 42, 'u': 43, '&': 44, '$': 45, 'x': 46, '%': 47, ':': 48, '@': 49, '^': 50, 'I': 51, 'L': 52, 'Z': 53, 'h': 54, 'W': 55, 'A': 56, 'v': 57, '?': 58, '2': 59, '~': 60, 's': 61, 'T': 62, 'R': 63, ',': 64, '|': 65, '4': 66, '>': 67, 'y': 68, '(': 69, '[': 70, 'k': 71, 'H': 72, 'l': 73, 'j': 74, '7': 75, 'n': 76, 'i': 77, 'D': 78, 'Q': 79, ' ': 80, 'm': 81, 'Y': 82, '*': 83, '}': 84, '#': 85, 'p': 86, 'q': 87, '5': 88, 'c': 89, '`': 90, 'a': 91, 'b': 92, 'w': 93, '3': 94, 'E': 95, ';': 96, '-': 97}
Input decoding {2: 'o', 3: '{', 4: '.', 5: 'J', 6: '0', 7: '1', 8: '<', 9: 'B', 10: 'd', 11: '£', 12: '

In [18]:
f_test = open("/data/users/romain.claret/tm/wikidata-simplequestions/annotated_wd_data_test.txt",'r')
out_test = f_test.readlines()

In [2]:
def pickle_data(df, filename):
    pd.set_option('display.max_columns', None)
    filename = "/data/users/romain.claret/tm/wikidata-simplequestions/benchmark_pickles/"+filename+'.pickle.bz2'
    #df.summary = df.summary.map(sanitize_str)
    print("Done!",filename)
    return df.to_pickle(filename, compression='bz2')

In [5]:
#lang: en, fr, de, it, es, zh
#kb: dbpedia, wikidata, dblp, freebase
def ask_qanswer(question):
    data = {'query': question,'lang': 'en','kb': 'wikidata'}
    query = requests.post('http://qanswer-core1.univ-st-etienne.fr/api/gerbil', data=data)
    if not query:
        return False
    if (query.json()['questions'][0]['question']['answers']) == None:
        return False
    #if (query.json()['questions'][0]['question']['answers'].replace('\n', '')) == None:
    #    return False
    #print(query.json()['questions'][0]['question']['answers'].replace('\n', '').get("results"))
    try:
        response = (json.loads(query.json()
                .get("questions")[0]
                .get("question")
                .get("answers")
                .replace('\n', ''))
         .get("results").get("bindings"))
    except:
        return False
    
    if response:
        return response[0].get("o1").get("value")[len("http://www.wikidata.org/entity/"):] if response[0].get("o1") is not None else False
    else:
        return False

#ask_qanswer("Who is the wife of Barack Obama")
#ask_qanwser("Which equestrian was born in dublin?")
#ask_qanswer("what is the main language spoken in a ghentar si muore facile")
#ask_qanswer("was the film helpmates in color or black-and-white?")

In [6]:
def ask_platypus(question):
    headers = {'Accept': 'application/json','Accept-Language': 'en',}
    params = (('q', question),('lang', 'en'))

    response = requests.get('https://qa.askplatyp.us/v0/ask', headers=headers, params=params)
    if response:
        if type(response.json()['member']) is list:
            #print(response.json()['member'][0]['result'])
            if response.json()['member'] != []:
                if '@id' in (json.dumps(response.json()['member'][0]['result'])):
                    ps_result = (json.dumps(response.json()['member'][0]['result']['@id']))
                else: return False
            else: return False
        else:
            try:
                if '@id' in (json.dumps(response.json()['member']['result'])):
                    ps_result = (json.dumps(response.json()["member"]['result']['@id']))
                else: return False
            except:
                return False
    else: return False
    ps_result = ps_result[4:-1]
    #print(result[:1])
    if ps_result[:1] != 'P' and ps_result[:1] != 'Q':
        return False
    return ps_result
#ask_platypus("Which genre of album is harder.....faster?")
#ask_platypus("how does engelbert zaschka identify")
#ask_platypus("Which Swiss conductor's cause of death is myoc...")
#ask_platypus("where was padraic mcguinness's place of death")
#ask_platypus("was the film helpmates in color or black-and-white?")
#ask_platypus("Who created the show life on earth")
#ask_platypus("Who is the wife of Barack Obama")

In [7]:
def ask_convex(question):
    cx_result = cx.answer_complete_question(question, cx.tagmeToken)['answers'][0]['answer']
    #print(cx_result)
    #answer = str(cx.wd.wikidata_id_to_label(result['answers'][0]['answer']))
    try:
        if not cx_result:
            return cx_result
        if cx_result[:1] != 'P' and cx_result[:1] != 'Q':
            return False
        return cx_result
    except:
        return False

#ask_convex("Which actor voiced the Unicorn in The Last Unicorn?")
#ask_convex("Which genre of album is harder.....faster?")
#ask_convex("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#ask_convex("Who is the wife of Barack Obama")
ask_convex("100% senorita is a television show in what language?")

Error: Did you put your tagMe_token in the settings file?
100% senorita is a television show in what language?


False

In [2]:
def ask_tmqa(question, verbose=False):
    tmqa_result = tmqa.answer_question(question, verbose=verbose)
    #print(tmqa_result)
    if not tmqa_result:
        return tmqa_result
    if tmqa_result[0][0][:1] != 'P' and tmqa_result[0][0][:1] != 'Q':
        return False
    return tmqa_result[0]

#answer = ask_tmqa_1("Which actor voiced the Unicorn in The Last Unicorn?", verbose=True)
#answer = ask_tmqa_1("what's akbar tandjung's ethnicity", verbose=True)
#ask_tmqa_1("Which genre of album is harder.....faster?")
#ask_tmqa_1("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#ask_tmqa_1("Who is the wife of Barack Obama")
#print(answer)

In [ ]:
# Evaluate qanswer
#arguments[0] Subject
#arguments[1] Predicate
#arguments[2] Object
#arguments[3] question

#HEADERS = ['question', 'source', 'qanswer', 'platypus', 'convex', 'tmqa']
#df = pd.DataFrame(columns=HEADERS)

start_time = time.time()
out_test_len = len(out_test)
#print(out_test_len)
for i, line in enumerate(out_test):
    arguments = line.split("\t")
    print("\r\t>>> Processing {}/{}: {}".format(i,out_test_len,arguments[3][:-1]), end='                                  ')
    if i < len(df):
        continue
    result_qanswer = ask_qanswer(arguments[3][:-1])
    result_platypus = ask_platypus(arguments[3][:-1])
    result_convex = ask_convex(arguments[3][:-1])
    #result_tmqa = ask_tmqa(arguments[3][:-1])[1]
    print("\n",i, "-> qAnswer", result_qanswer) if result_qanswer == arguments[2] else False
    print("\n",i, "-> Platypus", result_platypus) if result_platypus == arguments[2] else False
    print("\n",i, "-> Convex", result_convex) if result_convex == arguments[2] else False
    #print("\n",i, "-> TMqa", result_tmqa) if result_tmqa == arguments[2] else False
    df = df.append({"question":arguments[3][:-1], 'source':arguments[2], 'qanswer':result_qanswer, 'platypus':result_platypus, 'convex':result_convex, 'tmqa1':result_tmqa}, ignore_index=True)

end_time = time.time()
print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
print(df)

	>>> Processing 4028/9961: who was born in cheboygan

	>>> Processing 5201/9961: 100% senorita is a television show in what language?                                  

Error: Did you put your tagMe_token in the settings file?
100% senorita is a television show in what language?
	>>> Processing 5203/9961: what is ben moses's nationality?                                              
 5203 -> Platypus Q30
	>>> Processing 5209/9961: who was lethal weapon edited by                                                             
 5209 -> qAnswer Q1715774

 5209 -> Convex Q1715774
	>>> Processing 5211/9961: Where did reiko sato die?                                                
 5211 -> Convex Q65
	>>> Processing 5221/9961: where was wim deetman born                                                             
 5221 -> qAnswer Q36600

 5221 -> Platypus Q36600

 5221 -> Convex Q36600
	>>> Processing 5222/9961: what gender is franklin hansen?                                  
 5222 -> qAnswer Q6581097

 5222 -> Convex Q6581097
	>>> Processing 5225/9961: what type of film is el madina?                                                   
 5225 -> qAnswer Q130232

	>>> Processing 5588/9961: what type of music is the album off the beaten track in?                                  
 5588 -> Convex Q11399
	>>> Processing 5589/9961: where was andrew hoffman born?                                  
 5589 -> Platypus Q501785

 5589 -> Convex Q501785
	>>> Processing 5592/9961: what nationality is homi adajania                                                     
 5592 -> qAnswer Q668

 5592 -> Convex Q668
	>>> Processing 5594/9961: Which position does john olufemi play?                                  
 5594 -> Convex Q280658
	>>> Processing 5595/9961: where in romania did ignat bednarik die?                                  
 5595 -> Convex Q19660
	>>> Processing 5596/9961: What is kevin cadogan profession?                                  
 5596 -> qAnswer Q177220
	>>> Processing 5600/9961: is mitchel mclaughlin male or female                                              
 5600 -> Convex Q6581097
	>>> Processing 5602/9961: what is herman of alaska's 

	>>> Processing 5964/9961: What nationality is chris kaman                                                                           
 5964 -> qAnswer Q183

 5964 -> Convex Q183
	>>> Processing 5976/9961: what is the sex of katie jarvis                                                                  
 5976 -> qAnswer Q6581072

 5976 -> Convex Q6581072
	>>> Processing 5979/9961: What gender is L.L Marshall?                                            
 5979 -> qAnswer Q6581097

 5979 -> Convex Q6581097
	>>> Processing 5986/9961: who was the director for the beaver?                                                                
 5986 -> qAnswer Q41351

 5986 -> Convex Q41351
	>>> Processing 5989/9961: Where was antun palić born?                                                        
 5989 -> Convex Q1435
	>>> Processing 5990/9961: Where was colin bettany born?                                  
 5990 -> Convex Q83065
	>>> Processing 5991/9961: what is the nationality of thabo nthethe   

...

	>>> Processing 6771/9961: What is the gender of carolyn a. brent?                                  
 6771 -> qAnswer Q6581072

 6771 -> Convex Q6581072
	>>> Processing 6776/9961: which male is patrick wintour                                                                   
 6776 -> Convex Q6581097
	>>> Processing 6779/9961: who are ras baraka's parents?                                         
 6779 -> qAnswer Q354783
	>>> Processing 6783/9961: what is the gameplay mode of the elder scrolls v: skyrim – hearthfire?                                  
 6783 -> qAnswer Q208850

 6783 -> Convex Q208850
	>>> Processing 6784/9961: Which company produced i'll be yours                                  
 6784 -> Convex Q168383
	>>> Processing 6785/9961: who is the artist of man on the moon: the end of day                                  
 6785 -> qAnswer Q299138
	>>> Processing 6794/9961: what genre is the fully down                                                                
 6794 -> qAn

	>>> Processing 7146/9961: What language is spoken in fargo                                  
 7146 -> qAnswer Q1860

 7146 -> Convex Q1860
	>>> Processing 7150/9961: What is hiram n. breed's nationality?                                            
 7150 -> qAnswer Q30

 7150 -> Convex Q30
	>>> Processing 7152/9961: which language is used in the vineyard man                                  
 7152 -> Convex Q9176
	>>> Processing 7154/9961: What is josé antonio remón cantera's profession?                                                 
 7154 -> qAnswer Q82955
	>>> Processing 7162/9961: what kind of music does tonina torrielli perform                                            
 7162 -> qAnswer Q37073
	>>> Processing 7177/9961: what country is jacques lisfranc de st. martin originally from?                                  
 7177 -> qAnswer Q142

 7177 -> Convex Q142
	>>> Processing 7180/9961: which musician recorded aerial ballet                                   
 7180 -> qAnswer Q281

	>>> Processing 7393/9961: What is angela ruggiero's gender?                                                    
 7393 -> qAnswer Q6581072

 7393 -> Convex Q6581072
	>>> Processing 7405/9961: what is the production company responsible for autumn marathon                                    
 7405 -> Convex Q141336
	>>> Processing 7408/9961: what is the gender of jagz kooner                                                  
 7408 -> qAnswer Q6581097

 7408 -> Convex Q6581097
	>>> Processing 7409/9961: what is the nationality of heinrich wilhelm olbers                                  
 7409 -> qAnswer Q183

 7409 -> Convex Q183
	>>> Processing 7411/9961: what position is josé della torre charged of                                                     
 7411 -> Convex Q336286
	>>> Processing 7414/9961: what is stanley fischer's ethnicity?                                                      
 7414 -> qAnswer Q7325

 7414 -> Convex Q7325
	>>> Processing 7416/9961: what is the best descripti

	>>> Processing 7756/9961: Which city was alexandra cassavetes born in                                  
 7756 -> Convex Q65
	>>> Processing 7757/9961: is eugen steinach male or female                                  
 7757 -> Convex Q6581097
	>>> Processing 7758/9961: What is mark barnes's gender                                  
 7758 -> qAnswer Q6581097

 7758 -> Convex Q6581097
	>>> Processing 7761/9961: What country was estanis gonzález from?                                    
 7761 -> qAnswer Q29
	>>> Processing 7763/9961: what gender is cesare dandini?                                                    
 7763 -> qAnswer Q6581097

 7763 -> Convex Q6581097
	>>> Processing 7764/9961: what caused manuel gayoso de lemos's death                                  
 7764 -> Convex Q154874
	>>> Processing 7767/9961: where did manuel montt pass away                                         
 7767 -> Convex Q2887
	>>> Processing 7770/9961: which language is in the shadow of the wind in    

	>>> Processing 7932/9961: What asteroid group is 6247 amanogawa a member of                                  
 7932 -> Convex Q2179
	>>> Processing 7933/9961: What is the gender of gaston bachelard                                  
 7933 -> qAnswer Q6581097

 7933 -> Convex Q6581097
	>>> Processing 7934/9961: what sex is raj singh chaudhary                                  
 7934 -> qAnswer Q6581097

 7934 -> Convex Q6581097
	>>> Processing 7935/9961: Is daria werbowy male or female                                  
 7935 -> Convex Q6581072
	>>> Processing 7936/9961: is héctor andrés negroni male or female                                  
 7936 -> Convex Q6581097
	>>> Processing 7942/9961: which country released beyond the rocks                                           
 7942 -> Convex Q30
	>>> Processing 7943/9961: Who was the composer for loca?                                  
 7943 -> qAnswer Q187844
	>>> Processing 7948/9961: What is the genre of the videogame hyper v-ball     

In [3]:
len(df)

NameError: name 'df' is not defined

In [17]:
df.tail(5)

,question,source,qanswer,platypus,convex,tm1,tm2
9956,who was the creator of the fictional character...,Q311607,False,False,Q28975,False,False
9957,what's a college sporting event that took plac...,Q3994958,False,False,Q10219353,False,False
9958,what celestial object is 2974 holden,Q3863,False,False,False,False,False
9959,what is the film genre for snow falling on ced...,Q1257444,Q1054574,False,Q130232,False,False
9960,what nationality is lucille clifton,Q30,Q30,False,Q30,False,False


In [16]:
#SAVING
pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-tm1-from-0-to-"+str(len(df_loaded)))

Done! /data/users/romain.claret/tm/wikidata-simplequestions/benchmark_pickles/benchmarking-qanswer-platypus-convex-tm1-from-0-to-9961.pickle.bz2


In [3]:
#LOADING
df_loaded = pd.read_pickle("/data/users/romain.claret/tm/wikidata-simplequestions/benchmark_pickles/benchmarking-qanswer-platypus-convex-tm1-from-0-to-9961.pickle.bz2")

In [4]:
#df = df_loaded

In [4]:
df_loaded

,question,source,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,False,55.4,False,False,False,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,Q319608,222.1,False,False,False,False,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,Q134556,693.1,False,False,False,False,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,Q19970531,112.23,False,False,False,False,False
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,False,333.3,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9956,who was the creator of the fictional character...,Q311607,False,False,Q28975,False,False,False,False,False,False,False
9957,what's a college sporting event that took plac...,Q3994958,False,False,Q10219353,False,False,False,False,False,False,False
9958,what celestial object is 2974 holden,Q3863,False,False,False,False,False,False,False,False,False,False
9959,what is the film genre for snow falling on ced...,Q1257444,Q1054574,False,Q130232,False,False,False,False,False,False,False


In [65]:
#df_loaded = df_loaded.replace("", False)

In [ ]:
#df_loaded['qanswer'][34] = False

In [ ]:
#del df_loaded['tm2']

In [31]:
#df_loaded.rename({'mine':'tm1'}, axis=1, inplace=True)

In [39]:
#df_loaded['tm1_top4'] = "False"

In [43]:
#df_loaded = df_loaded[['question','source','qanswer','platypus','convex','tm1','tm1_time','tm1_top2','tm1_top3','tm1_top4','tm1_top5','tm1_topall']]

In [66]:
#df_loaded

,question,source,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,Q544440,3558.7,False,False,False,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,Q445559,723.63,False,False,False,False,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,False,1846.62,False,False,False,False,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,Q1637790,249.12,False,False,False,False,False
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,Q1394435,1710.05,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9956,who was the creator of the fictional character...,Q311607,False,False,Q28975,False,False,False,False,False,False,False
9957,what's a college sporting event that took plac...,Q3994958,False,False,Q10219353,False,False,False,False,False,False,False
9958,what celestial object is 2974 holden,Q3863,False,False,False,False,False,False,False,False,False,False
9959,what is the film genre for snow falling on ced...,Q1257444,Q1054574,False,Q130232,False,False,False,False,False,False,False


In [5]:
print(df_loaded.dtypes)

question      object
source        object
qanswer       object
platypus      object
convex        object
tm1           object
tm1_time      object
tm1_top2      object
tm1_top3      object
tm1_top4      object
tm1_top5      object
tm1_topall    object
dtype: object


In [ ]:
df_loaded_len = len(df_loaded)
for i, question in enumerate(df_loaded['question']):
    if i >= 0:
    #if i >= 497:
        source = str(df_loaded['source'][i])
        print(str(i)+"/"+str(df_loaded_len),question,"-> source:",source)
        
        start_time = time.time()
        result_tmqa_1 = ask_tmqa_1(question, verbose=True)
        
        if result_tmqa_1:
            df_loaded['tm1'][i] = result_tmqa_1[0]
            if source in result_tmqa_1[:2]:
                df_loaded['tm1_top2'][i] = True
            if source in result_tmqa_1[:3]:
                df_loaded['tm1_top3'][i] = True
            if source in result_tmqa_1[:4]:
                df_loaded['tm1_top4'][i] = True
            if source in result_tmqa_1[:5]:
                df_loaded['tm1_top5'][i] = True
            if source in result_tmqa_1:
                df_loaded['tm1_topall'][i] = True
        else:
            df_loaded['tm1'][i] = False
        end_time = time.time()
        df_loaded['tm1_time'][i] = round(end_time-start_time,2)
        print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
        print(str(str(df_loaded['tm1'][i])==str(source)),"---> result_tmqa_1:",str(result_tmqa_1))
        
        print(df_loaded.loc[i,:])
        print("\n")

0/9961 Which genre of album is harder.....faster? -> source: Q1096120
Auto correcting question: Which genre of album is harder.....faster?

-> q_nlp: Which genre of album is harder ... Faster 
-> q_themes: ([(album, ['Q1173065', 'Q482994', 'Q10666342']), (Genre, ['Q15758743', 'Q28490034', 'Q37446719', 'Q50913877']), (Album, ['Q11957254', 'Q11850638', 'Q10404408', 'Q6172625']), (genre, ['P415', 'P136', 'Q483394', 'Q188451'])], [Which genre, Which genre of album is harder ... Faster])
-> q_themes_enhanced: [('Harder ... Faster', ['Q5487302']), ('Faster', ['Q1180836']), ('Harder', ['Q12315707', 'Q1542625']), ('hard', ['Q5655097', 'Q47777555']), ('Hard', ['Q16796569', 'Q21515488'])]
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
-> q_predicates: [(be, ['P31']), (genre, ['P415', 'P136']), (album, ['P436', 'P1954']), (harder, ['P400', 'P1088'])]
-> q_focused_parts: [(genre, ['P415', 'P136', 'Q483394', 'Q188451']), (album, ['Q1173065', 'Q482994', 'Q10666

In [8]:
df_loaded.head(165)

,question,source,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,Q4104520,299.03,False,False,False,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,Q319608,222.1,False,False,False,False,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,Q134556,693.1,False,False,False,False,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,Q19970531,112.23,False,False,False,False,False
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,False,333.3,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
160,which publisher was behind age of conan: unch...,Q918638,Q1406,False,Q679933,False,450.98,False,False,False,False,False
161,of what nationality is ken mcgoogan,Q16,Q114,False,Q16,Q38220538,206.45,False,False,False,False,False
162,what sort of film is combat report,Q1935609,Q1935609,False,False,False,153.74,False,False,False,False,False
163,Name a person who died from bleeding.,Q558844,False,False,Q11466679,False,370.08,False,False,False,False,False


In [5]:
df_loaded.head(165)

,question,source,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,False,55.4,False,False,False,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,Q319608,222.1,False,False,False,False,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,Q134556,693.1,False,False,False,False,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,Q19970531,112.23,False,False,False,False,False
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,False,333.3,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
160,which publisher was behind age of conan: unch...,Q918638,Q1406,False,Q679933,False,450.98,False,False,False,False,False
161,of what nationality is ken mcgoogan,Q16,Q114,False,Q16,Q38220538,206.45,False,False,False,False,False
162,what sort of film is combat report,Q1935609,Q1935609,False,False,False,153.74,False,False,False,False,False
163,Name a person who died from bleeding.,Q558844,False,False,Q11466679,False,370.08,False,False,False,False,False


In [16]:
df = df_loaded.copy()
#df = df.replace("", False)

In [17]:
df_backup = df

In [18]:
max_row = 496
df_len = len(df)
df_qanswer_max = df[(df.index<=max_row) & (df.qanswer == df.source)]
df_qanswer_max_len = len(df_qanswer_max)

df_platypus_max = df[(df.index<=max_row) & (df.platypus == df.source)]
df_platypus_max_len = len(df_platypus_max)

df_convex_max = df[(df.index<=max_row) & (df.convex == df.source)]
df_convex_max_len = len(df_convex_max)

df_tm1_max = df[(df.index<=max_row) & (df.tm1 == df.source)]
df_tm1_max_len = len(df_tm1_max)

df_tm1_max_top2 = df[(df.index<=max_row) & (df.tm1_top2 == True)]
df_tm1_max_top2_len = len(df_tm1_max_top2)

df_tm1_max_top3 = df[(df.index<=max_row) & (df.tm1_top3 == True)]
df_tm1_max_top3_len = len(df_tm1_max_top3)

df_tm1_max_top4 = df[(df.index<=max_row) & (df.tm1_top4 == True)]
df_tm1_max_top4_len = len(df_tm1_max_top4)

df_tm1_max_top5 = df[(df.index<=max_row) & (df.tm1_top5 == True)]
df_tm1_max_top5_len = len(df_tm1_max_top5)

df_tm1_max_topall = df[(df.index<=max_row) & (df.tm1_topall == True)]
df_tm1_max_topall_len = len(df_tm1_max_topall)

print("qanswer:", df_qanswer_max_len,df_qanswer_max_len/max_row)
print("platypus:", df_platypus_max_len, df_platypus_max_len/max_row)
print("convex:", df_convex_max_len, df_convex_max_len/max_row)
print("tm1:", df_tm1_max_len, df_tm1_max_len/max_row)
print("tm1_top2:", df_tm1_max_top2_len, df_tm1_max_top2_len/max_row)
print("tm1_top3:", df_tm1_max_top3_len, df_tm1_max_top3_len/max_row)
print("tm1_top4:", df_tm1_max_top4_len, df_tm1_max_top4_len/max_row)
print("tm1_top5:", df_tm1_max_top5_len, df_tm1_max_top5_len/max_row)
print("tm1_topall:", df_tm1_max_topall_len, df_tm1_max_topall_len/max_row)

#df[ & (df.qanswer == df.source)]

qanswer: 77 0.15524193548387097
platypus: 5 0.010080645161290322
convex: 116 0.23387096774193547
tm1: 10 0.020161290322580645
tm1_top2: 24 0.04838709677419355
tm1_top3: 31 0.0625
tm1_top4: 34 0.06854838709677419
tm1_top5: 36 0.07258064516129033
tm1_topall: 48 0.0967741935483871


In [8]:
print("qanswer:",len(df[df.qanswer == df.source]),len(df[df.qanswer == df.source])/len(df))
print("platypus:",len(df[df.platypus == df.source]),len(df[df.platypus == df.source])/len(df))
print("convex:",len(df[df.convex == df.source]),len(df[df.convex == df.source])/len(df))
print("tm1:",len(df[df.tm1 == df.source]),len(df[df.tm1 == df.source])/len(df))
print("tm1_top2:",len(df[df.tm1_top2 == True]),len(df[df.tm1_top2 == True])/len(df))
print("tm1_top3:",len(df[df.tm1_top3 == True]),len(df[df.tm1_top3 == True])/len(df))
print("tm1_top4:",len(df[df.tm1_top4 == True]),len(df[df.tm1_top4 == True])/len(df))
print("tm1_top5:",len(df[df.tm1_top5 == True]),len(df[df.tm1_top5 == True])/len(df))
print("tm1_topall:",len(df[df.tm1_topall == True]),len(df[df.tm1_topall == True])/len(df))

qanswer: 1669 0.1675534584881036
platypus: 63 0.006324666198172874
convex: 2352 0.23612087139845397
tm1: 13 0.0013050898504166249
tm1_top2: 19 0.0019074390121473748
tm1_top3: 27 0.0027105712277883747
tm1_top4: 30 0.0030117458086537496
tm1_top5: 34 0.0034133119164742494
tm1_topall: 46 0.00461801023993575


In [62]:
df.tm1_top2 = df.tm1_top3